In [ ]:
%pip install -r requirements.txt
%pip install pandas
%pip install kaleido
%pip install plotly
%pip insall scipy
%pip install sklearn
%pip install tqdm

In [1]:
import numpy as np
import pandas as pd
import torch
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler


test_df = pd.read_csv('dataset/processed/test.csv',parse_dates=['timestamp'])  
train_df = pd.read_csv('dataset/processed/training.csv',parse_dates=['timestamp'])
validation_df = pd.read_csv('dataset/processed/validation.csv',parse_dates=['timestamp'])


test_df['timestamp']=pd.to_datetime(test_df['timestamp'],unit='s')
test_df['month'] = test_df['timestamp'].dt.month
test_df['hour'] = test_df['timestamp'].dt.hour
test_df['minute'] = test_df['timestamp'].dt.minute
test_df['day'] = test_df['timestamp'].dt.day
test_df['diff'] = test_df['value'] - test_df['predicted']

train_df['timestamp']=pd.to_datetime(train_df['timestamp'],unit='s')
train_df['month'] = train_df['timestamp'].dt.month
train_df['hour'] = train_df['timestamp'].dt.hour
train_df['minute'] = train_df['timestamp'].dt.minute
train_df['day'] = train_df['timestamp'].dt.day
train_df['diff'] = train_df['value'] - train_df['predicted']

validation_df['timestamp']=pd.to_datetime(validation_df['timestamp'],unit='s')
validation_df['month'] = validation_df['timestamp'].dt.month
validation_df['hour'] = validation_df['timestamp'].dt.hour
validation_df['minute'] = validation_df['timestamp'].dt.minute
validation_df['day'] = validation_df['timestamp'].dt.day
validation_df['diff'] = validation_df['value'] - validation_df['predicted']

split = np.array_split(validation_df, 2)

validation_df_error_means = split[0]
validation_df_classification = split[1]

import plotly.graph_objects as go

torch.set_default_dtype(torch.float64)

In [59]:
anomalies = train_df.query("is_anomaly==1")
normal = train_df.query("is_anomaly==0")

trace1 = go.Scatter(
    x = anomalies.timestamp,
    y = anomalies["diff"],
    mode='markers',
    name = 'anomalies'
)

trace2 = go.Scatter(
    x = normal.timestamp,
    y = normal["diff"],
    mode='markers',
    name = 'Ground Truth'
)

layout = go.Layout(
    title = 'Signal Value Plot',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1,trace2], layout=layout)
fig.show()


In [2]:
class AnomalyDetectionDataset(torch.utils.data.Dataset):

    def __init__(self, data, n_past=14, n_future=1,config=[]):
        self.n_future = n_future
        self.n_past = n_past
        self.x = []
        self.actual = []

        data_columns = ['value', 'predicted','month','hour','minute','day','diff']

        scaler = StandardScaler()
        
        x_scaled = data[data_columns].astype(float)
        scaler = scaler.fit(x_scaled)
        #print(type(x_scaled))
        #print(x_scaled.shape)
        x_scaled = scaler.transform(x_scaled)
        #print(type(x_scaled))
        #print(x_scaled.shape)

        self.X = []
        self.Y = []
        self.IsAnomaly = []
        data_np = data.to_numpy()
        #print(type(x_scaled))
        #print(type(data_np.astype(float).to_numpy()))
        
        SKIP_ANOMALIES = False
        if "SKIP_ANOMALIES" in config:
            SKIP_ANOMALIES = True

        for i in range(self.n_past, len(x_scaled) - self.n_future + 1):
            anomalies =  data_np[i:i + self.n_future, 2].astype(float)

            if float(1) in anomalies and SKIP_ANOMALIES:   ## Dont predict anomalies
                continue

            self.X.append(  torch.tensor(x_scaled[i - self.n_past:i, :])  )
            self.Y.append(  torch.tensor(x_scaled[i: i + self.n_future, : ])  )
            self.IsAnomaly.append(torch.tensor(anomalies) )

        assert len(self.X) == len(self.Y)

    def __len__(self):
        return len(self.X)


    def __getitem__(self, idx):
        return {
            "x": self.X[idx],
            "y": self.Y[idx],
            "is_anomaly": self.IsAnomaly[idx]
        }


def collate_batch(batch):
    x = []
    y = []
    is_anomaly = []

    for item in batch:
        x.append(item["x"])
        y.append(item["y"])
        is_anomaly.append(item["is_anomaly"])

    x = torch.nn.utils.rnn.pad_sequence(x, batch_first=True)
    y = torch.nn.utils.rnn.pad_sequence(y, batch_first=True)
    is_anomaly = torch.nn.utils.rnn.pad_sequence(is_anomaly, batch_first=True)

    return {
        "x": x,
        "y": y,
        "is_anomaly": is_anomaly
    }

In [ ]:
td = AnomalyDetectionDataset(validation_df_classification, n_past=14, n_future=1)
td[0]


In [ ]:
td[1]

In [ ]:
len(td)

In [3]:
import torch.nn as nn
import torch

class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class EncoderDecoderConvLSTM(nn.Module):
    def __init__(self, nf, in_chan,height):
        super(EncoderDecoderConvLSTM, self).__init__()

        """ ARCHITECTURE 

        # Encoder (ConvLSTM)
        # Encoder Vector (final hidden state of encoder)
        # Decoder (ConvLSTM) - takes Encoder Vector as input
        # Decoder (3D CNN) - produces regression predictions for our model

        """
        self.height = height
        self.encoder_1_convlstm = ConvLSTMCell(input_dim=in_chan,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.encoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_1_convlstm = ConvLSTMCell(input_dim=nf,  # nf + 1
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_CNN = nn.Conv3d(in_channels=nf,
                                     out_channels=1,
                                     kernel_size=(1, self.height+2, 3),
                                     padding=(0, 1, 1))
        # self.fc1 = nn.Linear(14 * 3 * batch_len, 24)
        # self.fc2 = nn.Linear(24, 12)
        # self.fc3 = nn.Linear(12, batch_len)


    def autoencoder(self, x, seq_len, future_step, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4):

        outputs = []

        # encoder
        for t in range(seq_len):
            h_t, c_t = self.encoder_1_convlstm(input_tensor=x[:, t, :, :],
                                               cur_state=[h_t, c_t])  # we could concat to provide skip conn here
            h_t2, c_t2 = self.encoder_2_convlstm(input_tensor=h_t,
                                                 cur_state=[h_t2, c_t2])  # we could concat to provide skip conn here

        # encoder_vector
        encoder_vector = h_t2

        # decoder
        for t in range(future_step):
            h_t3, c_t3 = self.decoder_1_convlstm(input_tensor=encoder_vector,
                                                 cur_state=[h_t3, c_t3])  # we could concat to provide skip conn here
            h_t4, c_t4 = self.decoder_2_convlstm(input_tensor=h_t3,
                                                 cur_state=[h_t4, c_t4])  # we could concat to provide skip conn here
            encoder_vector = h_t4
            outputs += [h_t4]  # predictions

        outputs = torch.stack(outputs, 1) # torch.Size([1, 1, nf, 14, 2])
        # print(outputs.shape)
        outputs = outputs.permute(0, 2, 1, 3, 4) #torch.Size([1, nf, 1, 14, 2])
        # print(outputs.shape)
        outputs = self.decoder_CNN(outputs) # torch.Size([1, 1, 1, nf, 2])
        # outputs = torch.flatten(outputs) # torch.Size([1, 1, 1, nf, 2])
        outputs = torch.nn.Sigmoid()(outputs)
        return outputs

    def forward(self, x, future_seq=0):

        """
        Parameters
        ----------
        input_tensor:
            5-D Tensor of shape (b, t, c, h, w)        #   batch, time, channel, height, width
        """

        # find size of different input dimensions
        b, seq_len, _, h, w = x.size()

        # initialize hidden states
        h_t, c_t = self.encoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t2, c_t2 = self.encoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t3, c_t3 = self.decoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t4, c_t4 = self.decoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))

        # autoencoder forward
        outputs = self.autoencoder(x, seq_len, future_seq, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4)

        return outputs

In [ ]:
##This is testing code
from itertools import cycle
from tqdm import trange



batch_len = 100
input_channels = 1 # 1
output_channels = 1 # 1
height = 14 ## Number of previous datapoints to use
width = 2 ## Number of features per datapoints


losses = {
    "train":[],
    "val":[]
}

train_dataset = AnomalyDetectionDataset(train_df, n_past=height, n_future=output_channels)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_len, shuffle=False, collate_fn=collate_batch
)

# use the first half of the normal dev data as the dev set for the LSTM
split = np.array_split(validation_df, 2)
val_data = split[0]
val_dataset = AnomalyDetectionDataset(val_data, n_past=height,n_future=output_channels)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_len, shuffle=False, collate_fn=collate_batch
)

val_data2 = split[1]
val_dataset2 = AnomalyDetectionDataset(val_data2, n_past=height,n_future=output_channels)
val_dataloader2 = torch.utils.data.DataLoader(
    val_dataset2, batch_size=batch_len, shuffle=False, collate_fn=collate_batch
)

epochs = 100

loss_function = torch.nn.MSELoss(reduction='sum')

len_train = len(train_dataloader)
len_val = len(val_dataloader)


# instantiate model
autoEncoder = EncoderDecoderConvLSTM(nf=64, in_chan=1, batch_len=batch_len)
optimizer = torch.optim.Adam(autoEncoder.parameters(), lr=0.001)

train_iter = cycle(train_dataloader)
val_iter = cycle(val_dataloader)

for epoch in trange(epochs):
    print("epoch", epoch)
    autoEncoder.train()
    plot_loss = []
    plot_val_loss = []

    
    for itr in range(len_train):

        batch = next(train_iter, None)
        loss = 0
        input = torch.tile(batch['x'],(2,1,1))[:batch_len,:,:]
        input = torch.unsqueeze(input, 1)
        input = torch.unsqueeze(input, 1)
        
        pred = autoEncoder(input, output_channels).squeeze()
        y = torch.tile(batch["y"],(2,1,1))[:batch_len,:].squeeze()
        loss = loss_function(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss =loss.item()/batch_len
        plot_loss.append(train_loss)
    print('epoch', epoch, 'train loss', sum(plot_loss)/len_train)
    losses["train"].append(plot_loss)
    autoEncoder.eval()
    for itr in range(len_val):
        
        batch = next(val_iter, None)
        loss=0
        input = torch.tile(batch['x'],(2,1,1))[:batch_len,:,:]
        input = torch.unsqueeze(input, 1)
        input = torch.unsqueeze(input, 1)

        pred = autoEncoder(input, output_channels).squeeze()

        y = torch.tile(batch["y"],(2,1,1))[:batch_len,:].squeeze()

        loss = loss_function(pred, y)
        val_loss = loss.item()/batch_len
        plot_val_loss.append(val_loss)
    print('epoch', epoch, 'val loss', sum(plot_val_loss)/len_val)
    losses["val"].append(plot_val_loss)
    
    

In [ ]:
##This is testing code

from functools import reduce

def mean(l):
    return sum(l)/len(l)

train_losses = list(map(mean, losses["train"]))
val_losses = list(map(mean, losses["val"]))


In [ ]:
##This is testing code

epochs = [i for i in range(100)]

trace1 = go.Scatter(
    x = epochs,
    y = train_losses,
    mode='lines',
    name = 'train losses'
)
trace2 = go.Scatter(
    x = epochs,
    y = val_losses,
    mode='lines',
    name = 'val losses'
)

layout = go.Layout(
    title = 'Trian vs Validation loss',
    xaxis = {'title' : "epoch"},
    yaxis = {'title' : "loss"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()


In [ ]:
##This is testing code

torch.save(autoEncoder.state_dict(), "models/convlstmae/100batch_100epoch_64hidden_14prev_1future_0_001lr_adam/model.pt")

In [ ]:
# reload the best model test
model = EncoderDecoderConvLSTM(nf=64, in_chan=1, batch_len=batch_len)
model.load_state_dict(torch.load("models/convlstmae/100batch_100epoch_64hidden_14prev_1future_0_001lr_adam/model.pt"))


In [11]:
def evaluate(model, dataloader, loss_function,params):
    """
    Returns error means, standard deviations, and loss
    """
    total_loss = 0
    model.eval()

    errors = torch.tensor([])
    batch_len = params["batch_len"]
    output_channels = params["output_channels"]


    with torch.no_grad():
        for batch in dataloader:
            loss = 0
            input = torch.tile(batch['x'],(batch_len,1,1))[:batch_len,:,:]
            input = torch.unsqueeze(input, 1)
            input = torch.unsqueeze(input, 1)


            pred = model(input, output_channels).squeeze()
            y = torch.tile(batch["y"],(batch_len,1,1))[:batch_len,:].squeeze()
            loss = loss_function(pred, y)
            
            print("pred", pred.shape)
            print("y", y.shape)
            print("input", input.shape)
            print("loss", loss.shape)

            batch_errors = y - pred
            if len(batch_errors.shape) == 1:
                batch_errors = torch.unsqueeze(batch_errors,0)
            if len(loss.shape) == 1:
                batch_errors = torch.unsqueeze(loss,0)
            
            #errors = torch.cat([errors, batch_errors])
            errors = torch.cat([errors, loss])

            total_loss += sum(loss)/batch_len

    avg_loss = total_loss / len(dataloader)
    print("Loss:", avg_loss)

    errors_np = errors.numpy()
    means = np.mean(errors_np, axis=0)
    stds = np.std(errors_np, axis=0)
    print("Error means:", means)
    print("Error standard deviations:", stds)

    print("errors", errors_np.shape)
    return errors_np, means, stds, avg_loss

In [5]:
from datetime import datetime;

def calculateF1Score(log_likelihoods, value_threshold, is_anomaly, params):
    """
    Precondition: log_likelihoods and is_anomaly should be sorted by predicted
    """
    pred_is_anomaly = log_likelihoods[:,0] < value_threshold

    is_anomaly = is_anomaly > 0
    tp = sum(pred_is_anomaly & is_anomaly)
    fp = sum(pred_is_anomaly & np.logical_not(is_anomaly))
    fn = sum(np.logical_not(pred_is_anomaly) & is_anomaly)

    best_f1 = tp / (tp + 1 / 2 * (fp + fn))
    best_predicted_threshold = log_likelihoods[0,1]

    for i in range(len(is_anomaly)):
        if pred_is_anomaly[i]:
            if is_anomaly[i]:
                tp -= 1
                fn += 1
            else:
                fp -= 1

            f1 = tp / (tp + 1 / 2 * (fp + fn))

            if f1 > best_f1:
                best_f1 = f1
                #best_predicted_threshold = log_likelihoods[i,1]

    return best_predicted_threshold, best_f1


def calculateF1ScoreFromPredictedThreshold(
    log_likelihoods, value_threshold, predicted_threshold, is_anomaly
):
    pred_is_anomaly = (
        (log_likelihoods[:,0] < value_threshold)
        & (log_likelihoods[:,1] >= predicted_threshold)
    )

    is_anomaly = is_anomaly > 0.5

    tp = sum(pred_is_anomaly & is_anomaly)
    fp = sum(pred_is_anomaly & np.logical_not(is_anomaly))
    fn = sum(np.logical_not(pred_is_anomaly) & is_anomaly)

    return tp / (tp + 1 / 2 * (fp + fn))


def calculateLogLikelihoods(model, dataloader, loss_function,params):
    errors_np, means, stds, avg_loss = evaluate(model, dataloader, loss_function,params)
    #return np.concatenate(([[0,0]], norm.logpdf(errors_np, loc=means, scale=stds))), errors_np, means, stds, avg_loss
    return norm.logpdf(errors_np, loc=means, scale=stds), errors_np, means, stds, avg_loss


def determineClassificationThresholds(model, dataloader, loss_function,params):
    """
    Returns the best f1, value threshold, and predicted threshold
    """
    best_f1 = -1
    batch_len = params["batch_len"]
    log_likelihoods, errors_np, means, stds, avg_loss = calculateLogLikelihoods(model, dataloader, loss_function,params)

    difference =  log_likelihoods[:,0] - log_likelihoods[:,1]
    difference = np.expand_dims(difference,axis=1)
    log_likelihoods = np.append(log_likelihoods, values=difference, axis=1)
    indices = np.argsort(log_likelihoods, axis=0)[:,0]

    is_anomaly = []

    for batch in dataloader:
        anomalies = torch.tile(batch['is_anomaly'],(batch_len,1))[:batch_len,:]
        is_anomaly += anomalies[:,0]
    is_anomaly = np.array(is_anomaly)
    
    # sort log likelihoods and anomalies by difference in value vs predicted
    log_likelihoods_sorted = log_likelihoods[indices]
    is_anomaly_sorted = is_anomaly[indices]

    #print(np.shape(log_likelihoods_sorted))
    #print(np.shape(np.expand_dims(is_anomaly_sorted,axis=1)))
    #print(np.shape(np.expand_dims(indices,axis=1)))

    deb = np.hstack((np.expand_dims(indices,axis=1),log_likelihoods_sorted,np.expand_dims(is_anomaly_sorted,axis=1)))
    #print(np.shape(deb))
    np.savetxt("foo{}.csv".format(datetime.now()), deb, delimiter=',')

    for value_threshold in log_likelihoods_sorted[:,0]:
        predicted_threshold, f1 = calculateF1Score(
            log_likelihoods_sorted, value_threshold, is_anomaly_sorted, params
        )

        if f1 > best_f1:
            best_f1 = f1
            best_value_threshold = value_threshold
            best_predicted_threshold = predicted_threshold

    print("Best f1:", best_f1)
    print("Best difference threshold:", best_value_threshold)
    print("Best predicted threshold:", best_predicted_threshold)

    f1 = calculateF1ScoreFromPredictedThreshold(
        log_likelihoods,
        best_value_threshold,
        best_predicted_threshold,
        is_anomaly,
    )
    print("Sanity check with method using predicted threshold:", f1)

    return best_f1, best_value_threshold, best_predicted_threshold


In [6]:
from tqdm import trange
from functools import reduce

def mean(l):
    return sum(l)/len(l)

def merge(l1, l2):
    return l1 + l2

def train(epochs, model,optimizer,loss_function, len_train, len_val, train_iter, val_iter, params):


    losses = {
        "train":[],
        "val":[]
    }

    batch_len = params["batch_len"]
    output_channels = params["output_channels"]

    plot_loss = []
    plot_val_loss = []

    model.eval()
    train_loss = 0
    val_loss = 0
    for itr in trange(len_train):
        batch = next(train_iter, None)
        loss=0
        input = torch.tile(batch['x'],(batch_len,1,1))[:batch_len,:,:]
        input = torch.unsqueeze(input, 1)
        input = torch.unsqueeze(input, 1)

        pred = model(input, params["output_channels"]).squeeze()

        print("pred shape:", pred.shape)

        y = torch.tile(batch["y"],(batch_len,1,1))[:batch_len,:].squeeze()

        loss = loss_function(pred, y)

        train_loss =loss.item()/batch_len
        plot_loss.append(train_loss)
    print('epoch', -1, 'train loss', sum(plot_loss)/len_train)
    losses["train"].append(plot_loss)

    for itr in trange(len_val):
        
        batch = next(val_iter, None)
        loss=0
        input = torch.tile(batch['x'],(batch_len,1,1))[:batch_len,:,:]
        input = torch.unsqueeze(input, 1)
        input = torch.unsqueeze(input, 1)

        pred = model(input, params["output_channels"]).squeeze()

        y = torch.tile(batch["y"],(batch_len,1,1))[:batch_len,:].squeeze()

        loss = loss_function(pred, y)
        val_loss = loss.item()/batch_len
        plot_val_loss.append(val_loss)

    print('epoch', -1, 'val loss', sum(plot_val_loss)/len_val)
    losses["val"].append(plot_val_loss)



    for epoch in range(epochs):
        model.train()
        plot_loss = []
        plot_val_loss = []
        
        
        for itr in trange(len_train):
            batch = next(train_iter, None)
            loss = 0
            input = torch.tile(batch['x'],(batch_len,1,1))[:batch_len,:,:]
            input = torch.unsqueeze(input, 1)
            input = torch.unsqueeze(input, 1)
            
            pred = model(input, output_channels).squeeze()
            y = torch.tile(batch["y"],(batch_len,1,1))[:batch_len,:].squeeze()
            loss = loss_function(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss =loss.item()/batch_len
            plot_loss.append(train_loss)
        print('epoch', epoch, 'train loss', sum(plot_loss)/len_train)
        losses["train"].append(plot_loss)

        model.eval()
        for itr in trange(len_val):
            
            batch = next(val_iter, None)
            loss=0
            input = torch.tile(batch['x'],(batch_len,1,1))[:batch_len,:,:]
            input = torch.unsqueeze(input, 1)
            input = torch.unsqueeze(input, 1)

            pred = model(input, params["output_channels"]).squeeze()

            y = torch.tile(batch["y"],(batch_len,1,1))[:batch_len,:].squeeze()

            loss = loss_function(pred, y)
            val_loss = loss.item()/batch_len
            plot_val_loss.append(val_loss)

        print('epoch', epoch, 'val loss', sum(plot_val_loss)/len_val)
        losses["val"].append(plot_val_loss)


    model_directory = "models/convlstmae/{}batchlen_{}epoch_{}hidden_{}prev_{}future_{}lr_adam" \
    .format(param["batch_len"], param["epochs"], param["nf"], param["height"], param["output_channels"], param["lr"])

    model_file_name = model_directory + "_model.pt"
    model_train_eval = model_directory + "_train_eval_plot.jpeg"
    model_losses = model_directory + "_loses"

    torch.save(model.state_dict(), model_file_name)


    def flatten(l):
        return [item for sublist in l for item in sublist]

    #train_losses = list(map(mean, losses["train"]))
    #val_losses = list(map(mean, losses["val"]))
    train_losses = flatten(losses["train"])
    val_losses = flatten(losses["val"])

    epochs = [i for i in range(epochs)]

    train_x = [i for i in range(len(train_losses))]
    val_x = [ i * (len(train_losses)//len(val_losses)) + (len(losses["train"][0]))  for i in range(len(val_losses))]

    #print(train_x)
    #print(val_x)

    trace1 = go.Scatter(
        x = train_x,
        y = train_losses,
        mode='lines',
        name = 'train losses'
    )

    trace2 = go.Scatter(
        x = val_x,
        y = val_losses,
        mode='lines',
        name = 'val losses'
    )

    layout = go.Layout(
        title = 'Trian vs Validation loss',
        xaxis = {'title' : "iteration"},
        yaxis = {'title' : "loss"}
    )
    fig = go.Figure(data=[trace1, trace2], layout=layout)

    fig.write_image(model_train_eval)


    return losses

In [21]:
from itertools import cycle

def grid(params,results):
    batch_len = params["batch_len"]
    epochs = params["epochs"]
    height = params["height"] ## Number of previous datapoints to use
    lr = params["lr"]
    nf = params["nf"]
    input_channels = 1 # 1
    output_channels = 1 # 1
    width = 2 ## Number of features per datapoints
    

    train_dataset = AnomalyDetectionDataset(train_df, n_past=height, n_future=output_channels,config=["SKIP_ANOMALIES"])
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_len, shuffle=False, collate_fn=collate_batch)

    val_dataset_error_means = AnomalyDetectionDataset(validation_df_error_means, n_past=height,n_future=output_channels,config=["SKIP_ANOMALIES"])
    val_dataloader_error_means = torch.utils.data.DataLoader(
        val_dataset_error_means, batch_size=batch_len, shuffle=False, collate_fn=collate_batch)

    val_dataset_classification = AnomalyDetectionDataset(validation_df_classification, n_past=height,n_future=output_channels)
    val_dataloader_classification = torch.utils.data.DataLoader(
        val_dataset_classification, batch_size=batch_len, shuffle=False, collate_fn=collate_batch)

    #loss_function = torch.nn.MSELoss(reduction='sum')
    loss_function = torch.nn.PoissonNLLLoss(reduction='mean')
    #loss_function = nn.NLLLoss(reduction='mean')
    #loss_function = nn.BCELoss(reduction='mean')



    len_train = len(train_dataloader)
    len_val = len(val_dataloader_error_means)

    autoEncoder = EncoderDecoderConvLSTM(nf=nf, in_chan=input_channels,height=height)
    optimizer = torch.optim.Adam(autoEncoder.parameters(), lr=lr)

    train_iter = cycle(train_dataloader)
    val_iter = cycle(val_dataloader_error_means)

    train(epochs, autoEncoder, optimizer, loss_function,  len_train, len_val, train_iter, val_iter, params)


    loss_function_2 = torch.nn.BCELoss(reduction='none')

    f1, value_threshold, predicted_threshold = determineClassificationThresholds(autoEncoder, val_dataloader_classification, loss_function_2, params)
    
    if f1 > results["best_f1_grid_search"] :
        results["best_f1_grid_search"] = f1
        results["best_model_grid_search"] = autoEncoder
        results["best_params_grid_search"] = params
        results["best_value_threshold_grid_search"] = value_threshold
        results["predicted_threshold_grid_search"] = predicted_threshold


In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    "height": list(reversed([7,14,21])),
    #"height": [],
    "lr":[0.0005,0.0001,0.01,0.005,0.1],
    "batch_len": list(reversed([16,8,4,2])),
    "nf": [64,128,256],
    "epochs": [5],
    "output_channels": [],
}

best_params = {
        "lr":0.001,
        "batch_len":100,
        "nf": 32,
        "height": 14,
        "epochs": 1,
        "output_channels": 1
}

for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    



In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[0.0005,0.0001,0.01,0.005,0.1],
    #"lr":[],
    #"batch_len": list(reversed([16,8,4,2])),
    #"batch_len": [],
    "nf": [64,128,256],
    #"epochs": [],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len":100,
        "nf": 32,
        "height": 14,
        "epochs": 1,
        "output_channels": 1
}

for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[0.0005,0.0001,0.01,0.005,0.1],
    #"lr":[],
    #"batch_len": list(reversed([16,4,2])),
    #"batch_len": [],
    #"nf": [],
    #"epochs": [],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len":100,
        "nf": 64,
        "height": 14,
        "epochs": 15,
        "output_channels": 1
}


param = best_params.copy()
print("params:", param)
losses = grid(param,results)


In [ ]:
params = {'lr': 0.0001, 'batch_len': 100, 'nf': 64, 'height': 14, 'epochs': 1, 'output_channels': 1}

val_dataset_classification_test = AnomalyDetectionDataset(validation_df_classification, n_past=14,n_future=1,scaler=scaler)
val_dataloader_classification_test = torch.utils.data.DataLoader(
    val_dataset_classification_test, batch_size=100, shuffle=False, collate_fn=collate_batch)

loss_function = torch.nn.MSELoss(reduction='sum')


# reload the best model

model = EncoderDecoderConvLSTM(nf=64, in_chan=1,height=14)

model.load_state_dict(torch.load("models/convlstmae/100batchlen_1epoch_64hidden_14prev_1future_0.0001lr_adam_model.pt"))

f1, value_threshold, predicted_threshold = determineClassificationThresholds(model, val_dataloader_classification_test, loss_function, params)


In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[0.0005,0.0001,0.01,0.005,0.1],
    #"lr":[],
    #"batch_len": list(reversed([16,4,2])),
    #"batch_len": [],
    #"nf": [],
    #"epochs": [],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len":1000,
        "nf": 64,
        "height": 14,
        "epochs": 5,
        "output_channels": 1
}


param = best_params.copy()
print("params:", param)
losses = grid(param,results)


In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[0.0005,0.0001,0.01,0.005,0.1],
    #"lr":[],
    #"batch_len": list(reversed([16,4,2])),
    #"batch_len": [],
    #"nf": [],
    #"epochs": [],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len":1500,
        "nf": 64,
        "height": 14,
        "epochs": 1,
        "output_channels": 1
}


param = best_params.copy()
print("params:", param)
losses = grid(param,results)


In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    "height": [7,14,21,35],
    "lr":[0.1,0.01,0.005,0.0001],
    #"lr":[],
    "batch_len": list(reversed([100,250,500,750])),
    #"batch_len": [],
    "nf": [32,64,128,256],
    #"epochs": [20],
    #"output_channels": [],
}

best_params = {
        "lr":0.0005,
        "batch_len":1000,
        "nf": 16,
        "height": 42,
        "epochs": 12,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

Best Params so far:  {'lr': 0.0001, 'batch_len': 1000, 'nf': 16, 'height': 21, 'epochs': 12, 'output_channels': 1}


In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    "height": [],
    "lr":[0.0001],
    "batch_len": list(reversed([100,250,500,750])),
    #"batch_len": [],
    "nf": [32,64,128,256],
    #"epochs": [20],
    #"output_channels": [],
}

best_params = {
        "lr":0.0005,
        "batch_len":1000,
        "nf": 16,
        "height": 21,
        "epochs": 12,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

Best Params so far:  {'lr': 0.0001, 'batch_len': 1000, 'nf': 16, 'height': 21, 'epochs': 12, 'output_channels': 1}
Optimized LR and height. 

In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[],
    "batch_len": [100,250,500,750],
    #"batch_len": [],
    "nf": [32,64,128,256],
    #"epochs": [20],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len":1000,
        "nf": 16,
        "height": 21,
        "epochs": 20,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

Best Params so far:  {'lr': 0.0001, 'batch_len': 750, 'nf': 16, 'height': 21, 'epochs': 12, 'output_channels': 1}
Optimized LR, height and batch_len

In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[],
    #"batch_len": [100,250,500,750],
    #"batch_len": [],
    "nf": [64,128,256,16],
    #"epochs": [20],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len":750,
        "nf": 32,
        "height": 21,
        "epochs": 20,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

Best f1: 0.6382978723404256
Best value threshold: -0.5378956928006757
Best predicted threshold: -110.40773312857297
Sanity check with method using predicted threshold: 0.6417112299465241
Best Params so far:  {'lr': 0.0001, 'batch_len': 750, 'nf': 64, 'height': 21, 'epochs': 20, 'output_channels': 1}

###USE Poisson Loss Now

In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[],
    #"batch_len": [100,250,500,750],
    #"batch_len": [],
    "nf": [16, 32,64,128,256],
    #"epochs": [5],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len":750,
        "nf": 256,
        "height": 21,
        "epochs": 20,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[],
    #"batch_len": [100,250,500,750],
    #"batch_len": [],
    "nf": list(reversed([128])),
    #"epochs": [5],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len": 775,
        "nf": 64,
        "height": 21,
        "epochs": 20,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

In [ ]:
params = {'lr': 0.0001, 'batch_len': 775, 'nf': 128, 'height': 21, 'epochs': 20, 'output_channels': 1}

val_dataset_classification_test = AnomalyDetectionDataset(validation_df_classification, n_past=21,n_future=1)
val_dataloader_classification_test = torch.utils.data.DataLoader(
    val_dataset_classification_test, batch_size=775, shuffle=False, collate_fn=collate_batch)

loss_function = torch.nn.PoissonNLLLoss(reduction='mean')

# reload the best model

model = EncoderDecoderConvLSTM(nf=128, in_chan=1,height=21)

model.load_state_dict(torch.load("models/convlstmae/775batchlen_20epoch_128hidden_21prev_1future_0.0001lr_adam_model.pt"))

f1, value_threshold, predicted_threshold = determineClassificationThresholds(model, val_dataloader_classification_test, loss_function, params)


In [ ]:
params = {'lr': 0.0001, 'batch_len': 775, 'nf': 128, 'height': 21, 'epochs': 20, 'output_channels': 1}

val_dataset_classification_test = AnomalyDetectionDataset(validation_df_classification, n_past=21,n_future=1)
val_dataloader_classification_test = torch.utils.data.DataLoader(
    val_dataset_classification_test, batch_size=775, shuffle=False, collate_fn=collate_batch)

loss_function = torch.nn.PoissonNLLLoss(reduction='mean')

# reload the best model

model = EncoderDecoderConvLSTM(nf=128, in_chan=1,height=21)

model.load_state_dict(torch.load("models/convlstmae/775batchlen_20epoch_128hidden_21prev_1future_0.0001lr_adam_model.pt"))

f1, value_threshold, predicted_threshold = determineClassificationThresholds(model, val_dataloader_classification_test, loss_function, params)


In [ ]:
df = pd.read_csv('models/convlstmae/LogLoss/foo2022-03-01 18:55:06.265419.csv')

anomalies = df.query("anomaly==1").sort_values(by='timestamp')
normal = df.query("anomaly==0").sort_values(by='timestamp')





anomaly_values = go.Scatter(
                    x=anomalies['timestamp'], y=anomalies['value'], # Data
                    mode='markers', name='anomalies values' 
                   )

anomaly_predicted = go.Scatter(
                    x=anomalies['timestamp'], y=anomalies['predicted'], # Data
                    mode='markers', name='anomalies predicted' 
                   )


normal_values = go.Scatter(
                    x=normal['timestamp'], y=normal['value'], # Data
                    mode='markers', name='normal values' 
                   )

normal_predicted = go.Scatter(
                    x=normal['timestamp'], y=normal['predicted'], # Data
                    mode='markers', name='normal predicted' 
                   )

layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[anomaly_values, anomaly_predicted, normal_values,normal_predicted], layout=layout)
fig.show()

# Plot data in the notebook
#py.iplot(fig, filename='simple-plot-from-csv')


In [ ]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[],
    #"batch_len": [100,250,500,750],
    #"batch_len": [],
    "nf": list(reversed([])),
    #"epochs": [5],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len": 775,
        "nf": 128,
        "height": 21,
        "epochs": 10,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

In [22]:
#params: {'lr': 0.0001, 'batch_len': 775, 'nf': 16, 'height': 21, 'epochs': 20, 'output_channels': 1}

params = {'lr': 0.0001, 'batch_len': 775, 'nf': 16, 'height': 21, 'epochs': 20, 'output_channels': 1}

val_dataset_classification_test = AnomalyDetectionDataset(validation_df_classification, n_past=21,n_future=1)
val_dataloader_classification_test = torch.utils.data.DataLoader(
    val_dataset_classification_test, batch_size=775, shuffle=False, collate_fn=collate_batch)

loss_function = nn.BCELoss(reduction='mean')

# reload the best model

model = EncoderDecoderConvLSTM(nf=1, in_chan=1,height=21)

model.load_state_dict(torch.load("models/convlstmae/time_feature/775batchlen_10epoch_128hidden_21prev_1future_0.0001lr_adam_model.pt"))

f1, value_threshold, predicted_threshold = determineClassificationThresholds(model, val_dataloader_classification_test, loss_function, params)


Loss: -0.0011937948257326769
Error means: [ 0.0202008   0.00690651 -0.0007887  -0.03019138  0.00626197  0.04071981]
Error standard deviations: [1.14154236 1.02884892 0.00782498 0.9877937  1.00435808 0.96255106]
errors (775, 6)
Best f1: 0.2702702702702703
Best difference threshold: 0.001466583663928267
Best predicted threshold: 0.3765428051598495
Sanity check with method using predicted threshold: 0.0


In [ ]:
#params: {'lr': 0.0001, 'batch_len': 775, 'nf': 16, 'height': 21, 'epochs': 20, 'output_channels': 1}

params = {'lr': 0.0001, 'batch_len': 775, 'nf': 128, 'height': 21, 'epochs': 10, 'output_channels': 1}

val_dataset_classification_test = AnomalyDetectionDataset(validation_df_classification, n_past=21,n_future=1)
val_dataloader_classification_test = torch.utils.data.DataLoader(
    val_dataset_classification_test, batch_size=775, shuffle=False, collate_fn=collate_batch)

loss_function = torch.nn.BCELoss(reduction='none')

# reload the best model

model = EncoderDecoderConvLSTM(nf=128, in_chan=1,height=21)

model.load_state_dict(torch.load("models/convlstmae/time_feature/775batchlen_10epoch_128hidden_21prev_1future_0.0001lr_adam_model.pt"))

f1, value_threshold, predicted_threshold = determineClassificationThresholds(model, val_dataloader_classification_test, loss_function, params)


In [49]:

df = pd.read_csv('foo2022-03-02 00:03:50.352694.csv')

anomalies = df.query("anomaly==1")
normal = df.query("anomaly==0")
anomalies.set_index("timestamp",inplace=True)
normal.set_index("timestamp",inplace=True)
normal.sort_values(by="value",inplace=True)
anomalies.sort_values(by="value",inplace=True)

anomalies['diff'] = anomalies['predicted'].subtract(anomalies['value'])
normal['diff'] = normal['predicted'].subtract(normal['value'])


anomaly_values = go.Scatter(
                    x=anomalies['diff'], y=anomalies['value'], # Data
                    mode='markers', name='anomalies values' 
                   )

#anomaly_predicted = go.Scatter(
#                    x=anomalies.index, y=anomalies['predicted'], # Data
#                    mode='markers', name='anomalies predicted' 
#                   )


normal_values = go.Scatter(
                    x=normal['diff'], y=normal['value'], # Data
                    mode='markers', name='normal values' 
                   )

#normal_predicted = go.Scatter(
#                    x=normal.index, y=normal['predicted'], # Data
#                    mode='markers', name='normal predicted' 
#                   )


anomaly_diff = go.Scatter(
                    x=anomalies.index, y=anomalies['diff'], # Data
                    mode='markers', name='anomalies diff' 
                   )


normal_diff = go.Scatter(
                    x=normal.index, y=normal['diff'], # Data
                    mode='markers', name='normal diff' 
                   )


layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

#fig = go.Figure(data=[anomaly_values, anomaly_predicted, normal_values,normal_predicted,anomaly_diff,normal_diff], layout=layout)
fig = go.Figure(data=[anomaly_values, normal_values], layout=layout)

fig.show()

# Plot data in the notebook
#py.iplot(fig, filename='simple-plot-from-csv')


/Users/bhatnaa/opt/miniconda3/envs/csep573/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bhatnaa/opt/miniconda3/envs/csep573/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[],
    #"batch_len": [100,250,500,750],
    #"batch_len": [],
    "nf": list(reversed([])),
    #"epochs": [5],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len": 775,
        "nf": 16,
        "height": 21,
        "epochs": 20,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

[16]
params: {'lr': 0.0001, 'batch_len': 775, 'nf': 16, 'height': 21, 'epochs': 20, 'output_channels': 1}


  6%|▋         | 1/16 [00:00<00:14,  1.01it/s]

pred shape: torch.Size([775, 7])


 12%|█▎        | 2/16 [00:02<00:15,  1.09s/it]

pred shape: torch.Size([775, 7])


...

100%|██████████| 16/16 [00:27<00:00,  1.70s/it]


epoch 19 train loss 0.001303612646873281


100%|██████████| 1/1 [00:00<00:00,  1.68it/s]


epoch 19 val loss 0.0013021707060345683
pred torch.Size([775, 7])
y torch.Size([775, 7])
input torch.Size([775, 1, 1, 21, 7])
loss torch.Size([])


RuntimeError: zero-dimensional tensor (at position 1) cannot be concatenated

In [22]:
#params: {'lr': 0.0001, 'batch_len': 775, 'nf': 16, 'height': 21, 'epochs': 20, 'output_channels': 1}

params = {'lr': 0.0001, 'batch_len': 775, 'nf': 128, 'height': 21, 'epochs': 10, 'output_channels': 1}

val_dataset_classification_test = AnomalyDetectionDataset(validation_df_classification, n_past=21,n_future=1)
val_dataloader_classification_test = torch.utils.data.DataLoader(
    val_dataset_classification_test, batch_size=775, shuffle=False, collate_fn=collate_batch)

loss_function = torch.nn.BCELoss(reduction='none')

# reload the best model

model = EncoderDecoderConvLSTM(nf=16, in_chan=1,height=21)

model.load_state_dict(torch.load("models/convlstmae/775batchlen_20epoch_16hidden_21prev_1future_0.0001lr_adam_model.pt"))

f1, value_threshold, predicted_threshold = determineClassificationThresholds(model, val_dataloader_classification_test, loss_function, params)


pred torch.Size([775, 7])
y torch.Size([775, 7])
input torch.Size([775, 1, 1, 21, 7])
loss torch.Size([775, 7])
Loss: tensor([-0.1336, -1.1521,  0.0015, -1.1146,  0.2232, -1.1924, -0.0709])
Error means: [-0.13358695 -1.15211759  0.00150392 -1.11461589  0.22319935 -1.19235298
 -0.07087463]
Error standard deviations: [3.56498053e+00 6.82454896e+00 2.39024449e-03 8.48772128e+00
 8.48040284e+00 7.03118144e+00 2.77541016e+00]
errors (775, 7)
Best f1: 0.5116279069767442
Best difference threshold: -2.3515221922930247
Best predicted threshold: -2.956721786698144
Sanity check with method using predicted threshold: 0.21875


In [27]:

df = pd.read_csv('foo2022-03-02 00:40:06.500164.csv')

anomalies = df.query("anomaly==1")
normal = df.query("anomaly==0")
anomalies.set_index("timestamp",inplace=True)
normal.set_index("timestamp",inplace=True)
normal.sort_values(by="diff",inplace=True)
anomalies.sort_values(by="diff",inplace=True)

#anomalies['diff'] = anomalies['predicted'].subtract(anomalies['value'])
#normal['diff'] = normal['predicted'].subtract(normal['value'])


anomaly_values = go.Scatter(
                    x=anomalies['value'], y=anomalies['diff'], # Data
                    mode='markers', name='anomalies values' 
                   )

#anomaly_predicted = go.Scatter(
#                    x=anomalies.index, y=anomalies['predicted'], # Data
#                    mode='markers', name='anomalies predicted' 
#                   )


normal_values = go.Scatter(
                    x=normal['value'], y=normal['diff'], # Data
                    mode='markers', name='normal values' 
                   )

#normal_predicted = go.Scatter(
#                    x=normal.index, y=normal['predicted'], # Data
#                    mode='markers', name='normal predicted' 
#                   )


anomaly_diff = go.Scatter(
                    x=anomalies.index, y=anomalies['diff'], # Data
                    mode='markers', name='anomalies diff' 
                   )


normal_diff = go.Scatter(
                    x=normal.index, y=normal['diff'], # Data
                    mode='markers', name='normal diff' 
                   )


layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

#fig = go.Figure(data=[anomaly_values, anomaly_predicted, normal_values,normal_predicted,anomaly_diff,normal_diff], layout=layout)
fig = go.Figure(data=[anomaly_values, normal_values], layout=layout)

fig.show()

# Plot data in the notebook
#py.iplot(fig, filename='simple-plot-from-csv')


In [28]:
results = {
    "best_f1_grid_search" : -float("inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    #"height": list(reversed([7,14,21])),
    #"height": [],
    #"lr":[],
    #"batch_len": [100,250,500,750],
    #"batch_len": [],
    "nf": list(reversed([])),
    #"epochs": [5],
    #"output_channels": [],
}

best_params = {
        "lr":0.0001,
        "batch_len": 775,
        "nf": 128,
        "height": 21,
        "epochs": 20,
        "output_channels": 1
}


for hyperparameter in params:
    param = best_params.copy()
    values = params[hyperparameter] + [best_params[hyperparameter]]
    print(values)
    for hyperparameter_value in  values:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

[128]
params: {'lr': 0.0001, 'batch_len': 775, 'nf': 128, 'height': 21, 'epochs': 20, 'output_channels': 1}


  6%|▋         | 1/16 [00:15<03:49, 15.33s/it]

pred shape: torch.Size([775, 7])


 12%|█▎        | 2/16 [00:32<03:49, 16.38s/it]

pred shape: torch.Size([775, 7])


...

epoch 18 val loss 0.001291132787316673


100%|██████████| 16/16 [13:19<00:00, 50.00s/it]


epoch 19 train loss 0.0012872026013830804


100%|██████████| 1/1 [00:10<00:00, 10.96s/it]


epoch 19 val loss 0.0012910935349921921
pred torch.Size([775, 7])
y torch.Size([775, 7])
input torch.Size([775, 1, 1, 21, 7])
loss torch.Size([775, 7])
Loss: tensor([-5.6923e-01, -1.9477e+00,  1.5225e-03, -4.1479e+00,  5.9934e-01,
        -2.0959e+00,  6.0463e-01])
Error means: [-5.69230399e-01 -1.94769781e+00  1.52252577e-03 -4.14788391e+00
  5.99335918e-01 -2.09594169e+00  6.04628261e-01]
Error standard deviations: [5.66369587e+00 9.68542481e+00 1.58848756e-02 2.16070213e+01
 2.25818671e+01 1.70124702e+01 1.41977030e+01]
errors (775, 7)
Best f1: 0.6329113924050633
Best difference threshold: -2.7082425795225733
Best predicted threshold: -10.976063326377824
Sanity check with method using predicted threshold: 0.5277777777777778
Best Params so far:  {'lr': 0.0001, 'batch_len': 775, 'nf': 128, 'height': 21, 'epochs': 20, 'output_channels': 1}


In [39]:

df = pd.read_csv('foo2022-03-02 05:31:04.060329.csv')

anomalies = df.query("anomaly==1")
normal = df.query("anomaly==0")
anomalies.set_index("timestamp",inplace=True)
normal.set_index("timestamp",inplace=True)
normal.sort_values(by="diff",inplace=True)
anomalies.sort_values(by="diff",inplace=True)

#anomalies['diff'] = anomalies['predicted'].subtract(anomalies['value'])
#normal['diff'] = normal['predicted'].subtract(normal['value'])


anomaly_values = go.Scatter(
                    x=anomalies['value'], y=anomalies['predicted'], # Data
                    mode='markers', name='anomalies values' 
                   )

#anomaly_predicted = go.Scatter(
#                    x=anomalies.index, y=anomalies['predicted'], # Data
#                    mode='markers', name='anomalies predicted' 
#                   )


normal_values = go.Scatter(
                    x=normal['value'], y=normal['predicted'], # Data
                    mode='markers', name='normal values' 
                   )

#normal_predicted = go.Scatter(
#                    x=normal.index, y=normal['predicted'], # Data
#                    mode='markers', name='normal predicted' 
#                   )


anomaly_diff = go.Scatter(
                    x=anomalies.index, y=anomalies['diff'], # Data
                    mode='markers', name='anomalies diff' 
                   )


normal_diff = go.Scatter(
                    x=normal.index, y=normal['diff'], # Data
                    mode='markers', name='normal diff' 
                   )


layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

#fig = go.Figure(data=[anomaly_values, anomaly_predicted, normal_values,normal_predicted,anomaly_diff,normal_diff], layout=layout)
fig = go.Figure(data=[anomaly_values, normal_values], layout=layout)

fig.show()

# Plot data in the notebook
#py.iplot(fig, filename='simple-plot-from-csv')


Evalutation on Test Data

In [33]:
#params: {'lr': 0.0001, 'batch_len': 775, 'nf': 128, 'height': 21, 'epochs': 20, 'output_channels': 1}

params = {'lr': 0.0001, 'batch_len': 775, 'nf': 128, 'height': 21, 'epochs': 10, 'output_channels': 1}

val_dataset_classification_test = AnomalyDetectionDataset(test_df, n_past=21,n_future=1)
val_dataloader_classification_test = torch.utils.data.DataLoader(
    val_dataset_classification_test, batch_size=775, shuffle=False, collate_fn=collate_batch)

loss_function = torch.nn.BCELoss(reduction='none')

# reload the best model

model = EncoderDecoderConvLSTM(nf=128, in_chan=1,height=21)

model.load_state_dict(torch.load("models/convlstmae/775batchlen_20epoch_128hidden_21prev_1future_0.0001lr_adam_model.pt"))

f1, value_threshold, predicted_threshold = determineClassificationThresholds(model, val_dataloader_classification_test, loss_function, params)


pred torch.Size([775, 7])
y torch.Size([775, 7])
input torch.Size([775, 1, 1, 21, 7])
loss torch.Size([775, 7])
pred torch.Size([775, 7])
y torch.Size([775, 7])
input torch.Size([775, 1, 1, 21, 7])
loss torch.Size([775, 7])
pred torch.Size([775, 7])
y torch.Size([775, 7])
input torch.Size([775, 1, 1, 21, 7])
loss torch.Size([775, 7])
Loss: tensor([-1.5983e+00, -3.9364e+00,  4.8091e-03, -1.6087e+01,  4.8623e-01,
         7.2708e+00, -7.0278e-01])
Error means: [-1.59831991e+00 -3.93636354e+00  4.80906111e-03 -1.60868501e+01
  4.86225677e-01  7.27079036e+00 -7.02779184e-01]
Error standard deviations: [ 4.47583956  9.95176776  0.05170062 24.67379922 22.89877581 21.90929112
  5.43638484]
errors (2325, 7)
Best f1: 0.6386554621848739
Best difference threshold: -3.895970101343442
Best predicted threshold: -3.468240301574455
Sanity check with method using predicted threshold: 0.2988505747126437
